<h1 style="color: green;">Monitoring reports generation</h1>
<p>
In this section monitoring reports are generated using EvidentlyAI preset functions.<br>
The following tasks are accomplished:<br>
<ul>
<li>Data quality for classifier model</li>
<li>Data quality for regressor model</li>
<li>Data drift report for classifier</li>
<li>Data drift report for regressor</li>
<li>Classification performance report: probability prediction</li>
<li>Regression performance report</li>
</ul>

The actual outputs can be viewed from "11. Reports" folder.
</p>

<h1 style="color: green;">Importing libraries</h1>

In [1]:
import evidently

import pandas as pd
import numpy as np
import copy

# from evidently
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import RegressionPreset
from evidently.metric_preset import ClassificationPreset
from evidently.metric_preset import TargetDriftPreset

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

<h1 style="color: green;">Load the classifier datasets</h1>

In [2]:
classifier_reference = pd.read_csv("../2. Data/classifier_reference.csv")

Classifier_curr_dt_no_drift = pd.read_csv("../2. Data/Classifier_curr_dt_no_drift.csv")

classifier_drift = pd.read_csv("../2. Data/classifier_current_dt_drift.csv")

In [3]:
classifier_reference.head()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,Trips_response_lt_5,prediction_label,prediction_prob
0,0.0,3,17.16,0.191725,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,0.999449
1,0.0,3,17.16,0.634615,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,0.999647
2,0.0,3,17.16,0.173660,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,0.999364
3,5.0,4,28.69,0.348554,7.1725,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1.0,0.999923
4,1.0,4,28.69,0.205995,7.1725,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1.0,0.999817


In [4]:
Classifier_curr_dt_no_drift.tail()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,Trips_response_lt_5,prediction_label,prediction_prob
6235,1.0,40,145.41,0.081012,3.635250,0,1,2,0,9,...,0,0,3,0,0,1,0,0,0.0,0.001244
6236,1.0,26,42.95,0.023050,1.651923,2,3,3,0,0,...,1,4,0,0,0,4,0,0,0.0,0.002121
6237,0.0,41,74.71,0.073618,1.822195,2,2,4,0,5,...,1,2,3,1,0,0,0,0,0.0,0.002415
6238,2.0,29,33.03,0.257342,1.138966,2,2,3,0,4,...,0,4,0,0,0,0,0,1,1.0,0.999640
6239,1.0,33,562.50,0.940444,17.045455,1,3,2,0,8,...,0,2,0,0,0,4,0,0,0.0,0.001070


In [5]:
classifier_drift.head()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,Trips_response_lt_5,prediction_label,prediction_prob
0,7.234651,16.842141,18.493836,0.250503,22.131373,10.455820,23.993976,6.934586,8.087692,10.944593,...,9.618020,2.513850,18.388972,6.810537,1.848405,22.149125,2.013632,1,1.0,0.411600
1,4.451534,39.957007,40.091199,0.437721,15.893993,6.858782,12.727666,11.008138,7.400027,4.847528,...,15.177531,4.460985,9.167653,19.762036,7.375986,9.772866,2.339759,0,1.0,0.502171
2,10.541442,41.863672,27.356854,0.541798,19.108762,16.057341,14.565109,20.475405,10.922261,3.023279,...,16.323361,5.686904,12.985114,26.249166,23.937695,22.769351,12.360392,0,1.0,0.475491
3,19.748735,70.364440,198.227742,0.378668,6.106105,12.873466,13.212764,14.628252,15.044239,7.889224,...,7.274041,0.937012,14.794072,12.485270,2.783435,10.189526,11.634937,0,1.0,0.365708
4,13.168847,41.559918,32.598282,0.206135,10.153430,6.515889,4.026499,16.951235,13.005726,23.996149,...,5.826471,15.394590,12.648097,9.358315,4.375565,4.381416,5.018939,0,1.0,0.533805


<h1 style="color: green;">Load the regressor datasets</h1>

In [6]:
regressor_reference = pd.read_csv("../2. Data/regressor_reference.csv")

regressor_curr_dt_no_drift = pd.read_csv("../2. Data/regressor_current_dt_no_drift3.csv")

regressor_drift = pd.read_csv("../2. Data/regressor_current_dt_drift.csv")

In [7]:
regressor_reference.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk,prediction
0,3.29,3,0.191725,0,0,0,0,0,1,0,0,0,17.16,18.844135
1,10.89,3,0.634615,0,0,0,0,0,1,0,0,0,17.16,21.880973
2,2.98,3,0.173660,0,0,0,0,0,1,0,0,0,17.16,17.159999
3,10.00,4,0.348554,0,0,0,0,0,1,0,0,0,28.69,29.611726
4,5.91,4,0.205995,0,0,0,0,0,1,0,0,0,28.69,28.690001


In [8]:
regressor_curr_dt_no_drift.tail()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk,prediction
6235,129.99,9,0.577990,2,7,2,0,0,1,0,0,2,224.90,224.931083
6236,3.50,5,0.037707,1,1,0,0,0,0,0,0,0,92.82,92.820008
6237,14.21,5,0.279669,1,3,0,0,0,0,0,1,0,50.81,50.809352
6238,6.85,9,0.053047,3,11,0,5,0,0,0,1,1,129.13,129.130001
6239,48.99,3,0.770525,2,6,0,0,0,0,0,0,0,63.58,72.761433


In [9]:
regressor_drift.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk,prediction
0,29.671353,27.339236,0.182824,21.605707,22.686535,14.965869,11.628342,11.374895,8.010144,17.332147,10.617962,3.965802,81.338213,103.036164
1,10.772184,16.613967,0.282498,5.329787,13.760860,6.712280,11.391198,5.400155,12.412875,11.662304,13.654401,15.852365,47.051797,133.756120
2,2.789794,8.747711,1.065763,14.982056,10.591974,4.179518,2.854285,13.613707,8.852314,9.874487,5.014989,14.859072,23.460449,113.995778
3,19.297759,29.517343,0.193383,20.215022,7.217432,5.672266,4.964222,9.193924,18.293691,6.078218,8.594319,14.789202,79.541401,118.684781
4,15.021405,30.802891,0.093229,5.633105,10.219041,6.554663,17.109600,6.630360,5.371829,8.588563,13.757232,21.658638,82.716934,119.798779


<h1 style="color: green;">Data quality for classifier model</h1><br>

<h3 style="color: green;">Classifier data preparation for data quality report</h3><br>
Note, for this the target and prediction are not required.<br>

In [10]:
# dropping the classifier target and prediction from: classifier_reference


classifier_reference1 = copy.deepcopy(classifier_reference)
classifier_reference1.drop(['Trips_response_lt_5', 
                            'prediction_label', 
                            'prediction_prob'], 
                           axis=1,
                           inplace=True)

In [11]:
# dropping the classifier target and prediction from: Classifier_curr_dt_no_drift

Classifier_curr_dt_no_drift1 = copy.deepcopy(Classifier_curr_dt_no_drift)
Classifier_curr_dt_no_drift1.drop(['Trips_response_lt_5', 
                            'prediction_label', 
                            'prediction_prob'], 
                           axis=1,
                           inplace=True)

In [12]:
# dropping the classifier target and prediction from: Classifier_curr_dt_no_drift

classifier_drift1 = copy.deepcopy(classifier_drift)
classifier_drift1.drop(['Trips_response_lt_5', 
                            'prediction_label', 
                            'prediction_prob'], 
                           axis=1,
                           inplace=True)

<h3 style="color: green;">Generating data quality reports for classifier data</h3><br>

In [13]:
# Data quality for classifier no drift
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=classifier_reference1, 
                        current_data=Classifier_curr_dt_no_drift1)
data_quality_report.save_html(
    "../11. Reports/evidently_metric_presets/data_quality/Classifier_No_drift_data_quality_report29082023.html")

In [14]:
# Data quality for classifier with drift
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=classifier_reference1, 
                        current_data=classifier_drift1)
data_quality_report.save_html(
    "../11. Reports/evidently_metric_presets/data_quality/Classifier_drift_data_quality_report29082023.html")

<h1 style="color: green;">Data quality for regressor model</h1><br>

<h3 style="color: green;">Regressor data preparation for data quality report</h3><br>
Note, for this the target and prediction are not required. However the target is one of the features<br>

In [15]:
regressor_reference1 = copy.deepcopy(regressor_reference) 

regressor_reference1.drop(['prediction'], axis=1, inplace=True)
regressor_reference1.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk
0,3.29,3,0.191725,0,0,0,0,0,1,0,0,0,17.16
1,10.89,3,0.634615,0,0,0,0,0,1,0,0,0,17.16
2,2.98,3,0.173660,0,0,0,0,0,1,0,0,0,17.16
3,10.00,4,0.348554,0,0,0,0,0,1,0,0,0,28.69
4,5.91,4,0.205995,0,0,0,0,0,1,0,0,0,28.69


In [16]:
regressor_curr_dt_no_drift1 = copy.deepcopy(regressor_curr_dt_no_drift)
regressor_curr_dt_no_drift1.drop(['prediction'], axis=1, inplace=True)
regressor_curr_dt_no_drift1.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk
0,10.00,10,0.144009,1,6,0,0,0,0,0,1,0,69.44
1,8.27,3,0.226948,3,9,0,0,0,0,1,0,0,36.44
2,1.60,1,1.000000,0,0,0,0,0,0,0,0,0,1.60
3,6.84,11,0.106992,7,6,1,0,0,0,0,3,0,63.93
4,4.93,9,0.084752,0,1,0,1,0,0,0,1,0,58.17


In [17]:
regressor_drift1 = copy.deepcopy(regressor_drift)
regressor_drift1.drop(['prediction'], axis=1, inplace=True)
regressor_drift1.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk
0,29.671353,27.339236,0.182824,21.605707,22.686535,14.965869,11.628342,11.374895,8.010144,17.332147,10.617962,3.965802,81.338213
1,10.772184,16.613967,0.282498,5.329787,13.760860,6.712280,11.391198,5.400155,12.412875,11.662304,13.654401,15.852365,47.051797
2,2.789794,8.747711,1.065763,14.982056,10.591974,4.179518,2.854285,13.613707,8.852314,9.874487,5.014989,14.859072,23.460449
3,19.297759,29.517343,0.193383,20.215022,7.217432,5.672266,4.964222,9.193924,18.293691,6.078218,8.594319,14.789202,79.541401
4,15.021405,30.802891,0.093229,5.633105,10.219041,6.554663,17.109600,6.630360,5.371829,8.588563,13.757232,21.658638,82.716934


<h3 style="color: green;">Generating data quality reports for regressor data</h3><br>

In [18]:
# Data quality for regressor no drift
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=regressor_reference1, 
                        current_data=regressor_curr_dt_no_drift1)
data_quality_report.save_html(
    "../11. Reports/evidently_metric_presets/data_quality/Regressor_No_drift_data_quality_report29082023.html")

In [19]:
# Data quality for regressor with drift
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=regressor_reference1, 
                        current_data=regressor_drift1)
data_quality_report.save_html(
    "../11. Reports/evidently_metric_presets/data_quality/Regressor_with_drift_data_quality_report29082023.html")

<h1 style="color: green;">Data drift report for classifier</h1>

In [20]:
# Data drift for classifier data with no drift
data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', 
                    cat_stattest='psi', 
                    num_stattest_threshold=0.19, 
                    cat_stattest_threshold=0.19),
])

data_drift_report.run(reference_data=classifier_reference1, 
                      current_data=Classifier_curr_dt_no_drift1)
data_drift_report.save_html("../11. Reports/evidently_metric_presets/data_drift/classifier_data_drift_curr_dt_no_drift_report29082023.html")

In [21]:
# Data drift for classifier data with drift
data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', 
                    cat_stattest='psi', 
                    num_stattest_threshold=0.19, 
                    cat_stattest_threshold=0.19),
])

data_drift_report.run(reference_data=classifier_reference1, 
                      current_data=classifier_drift1)

data_drift_report.save_html("../11. Reports/evidently_metric_presets/data_drift/classifier_data_drift_curr_dt_with_drift_report29082023.html")

<h1 style="color: green;">Data drift report for regressor</h1>

In [22]:
# Data drift for classifier data with no drift
data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', 
                    cat_stattest='psi', 
                    num_stattest_threshold=0.19, 
                    cat_stattest_threshold=0.19),
])

data_drift_report.run(reference_data=regressor_reference1, 
                      current_data=regressor_curr_dt_no_drift1)
data_drift_report.save_html("../11. Reports/evidently_metric_presets/data_drift/regressor_data_drift_curr_dt_no_drift_report29082023.html")

In [23]:
# Data drift for classifier data with drift
data_drift_report = Report(metrics=[
    DataDriftPreset(num_stattest='ks', 
                    cat_stattest='psi', 
                    num_stattest_threshold=0.19, 
                    cat_stattest_threshold=0.19),
])

data_drift_report.run(reference_data=regressor_reference1, 
                      current_data=regressor_drift1)

data_drift_report.save_html("../11. Reports/evidently_metric_presets/data_drift/regressor_data_drift_curr_dt_with_drift_report29082023.html")

<h1 style="color: green;">Classification performance report: probability prediction</h1>

In [24]:
classifier_reference.head()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,Trips_response_lt_5,prediction_label,prediction_prob
0,0.0,3,17.16,0.191725,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,0.999449
1,0.0,3,17.16,0.634615,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,0.999647
2,0.0,3,17.16,0.173660,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.0,0.999364
3,5.0,4,28.69,0.348554,7.1725,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1.0,0.999923
4,1.0,4,28.69,0.205995,7.1725,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1.0,0.999817


In [25]:
# dropping prediction_label 
# renaming: prediction_prob to prediction, Trips_response_lt_5 to target
classifier_reference3 = copy.deepcopy(classifier_reference)
classifier_reference3.drop(['prediction_label'], axis=1, inplace=True)
classifier_reference3.rename({'Trips_response_lt_5':'target',
                              'prediction_prob':'prediction'
                             },axis=1, inplace=True)
classifier_reference3.head()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Fruit_wk,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,target,prediction
0,0.0,3,17.16,0.191725,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.999449
1,0.0,3,17.16,0.634615,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.999647
2,0.0,3,17.16,0.173660,5.7200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.999364
3,5.0,4,28.69,0.348554,7.1725,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0.999923
4,1.0,4,28.69,0.205995,7.1725,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0.999817


In [26]:
# dropping prediction_label 
# renaming: prediction_prob to prediction, Trips_response_lt_5 to target
Classifier_curr_dt_no_drift3 = copy.deepcopy(Classifier_curr_dt_no_drift)
Classifier_curr_dt_no_drift3.drop(['prediction_label'], axis=1, inplace=True)
Classifier_curr_dt_no_drift3.rename({'Trips_response_lt_5':'target',
                              'prediction_prob':'prediction'
                             },axis=1, inplace=True)
Classifier_curr_dt_no_drift3.head()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Fruit_wk,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,target,prediction
0,0.0,3,17.16,0.191725,5.720000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.999449
1,3.0,31,33.16,0.309409,1.069677,3,2,3,0,1,...,1,0,2,0,0,0,1,2,0,0.810571
2,2.0,20,22.69,0.484354,1.134500,4,0,2,0,1,...,1,0,0,0,0,0,5,0,0,0.006826
3,0.0,63,175.43,0.374508,2.784603,2,2,5,0,6,...,1,0,0,0,0,0,3,1,0,0.000316
4,2.0,31,29.12,0.154190,0.939355,1,1,2,1,8,...,0,0,1,0,0,0,1,1,0,0.030508


In [27]:
classification_performance_report = Report(metrics=[
    ClassificationPreset(probas_threshold=0.1423),
])

classification_performance_report.run(reference_data=classifier_reference3, current_data=Classifier_curr_dt_no_drift3)

classification_performance_report.save_html("../11. Reports/evidently_metric_presets/model_performance/classifier_curr_dt_no_drift_model_performance_report29082023.html")

In [28]:
# dropping prediction_label 
# renaming: prediction_prob to prediction, Trips_response_lt_5 to target
classifier_drift3 = copy.deepcopy(classifier_drift)
classifier_drift3.drop(['prediction_label'], axis=1, inplace=True)
classifier_drift3.rename({'Trips_response_lt_5':'target',
                              'prediction_prob':'prediction'
                             },axis=1, inplace=True)
classifier_drift3.head()

,Date_diff,Nbr_items_per_wk,Expenditure_per_wk,Total_Exp_wk_perc,AVG_exp_item_per_wk,Bread_wk,Cooked_meats_wk,Raw_meats_wk,Eating_out_wk,Snacks_wk,...,Fruit_wk,Cooking_base_wk,Dairy_produce_wk,Breakfast_wk,Transport_wk,Tech_and_services_wk,Cosmetics_and_selfcare_wk,House_and_kitchen_wk,target,prediction
0,7.234651,16.842141,18.493836,0.250503,22.131373,10.455820,23.993976,6.934586,8.087692,10.944593,...,2.263492,9.618020,2.513850,18.388972,6.810537,1.848405,22.149125,2.013632,1,0.411600
1,4.451534,39.957007,40.091199,0.437721,15.893993,6.858782,12.727666,11.008138,7.400027,4.847528,...,14.372260,15.177531,4.460985,9.167653,19.762036,7.375986,9.772866,2.339759,0,0.502171
2,10.541442,41.863672,27.356854,0.541798,19.108762,16.057341,14.565109,20.475405,10.922261,3.023279,...,12.175663,16.323361,5.686904,12.985114,26.249166,23.937695,22.769351,12.360392,0,0.475491
3,19.748735,70.364440,198.227742,0.378668,6.106105,12.873466,13.212764,14.628252,15.044239,7.889224,...,24.174820,7.274041,0.937012,14.794072,12.485270,2.783435,10.189526,11.634937,0,0.365708
4,13.168847,41.559918,32.598282,0.206135,10.153430,6.515889,4.026499,16.951235,13.005726,23.996149,...,8.537511,5.826471,15.394590,12.648097,9.358315,4.375565,4.381416,5.018939,0,0.533805


In [29]:
classification_performance_report = Report(metrics=[
    ClassificationPreset(probas_threshold=0.1423),
])

classification_performance_report.run(reference_data=classifier_reference3, current_data=classifier_drift3)

classification_performance_report.save_html("../11. Reports/evidently_metric_presets/model_performance/classifier_curr_dt_with_drift_model_performance_report29082023.html")

<h1 style="color: green;">Regression performance report</h1>

In [30]:
regressor_reference.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,Expenditure_per_wk,prediction
0,3.29,3,0.191725,0,0,0,0,0,1,0,0,0,17.16,18.844135
1,10.89,3,0.634615,0,0,0,0,0,1,0,0,0,17.16,21.880973
2,2.98,3,0.173660,0,0,0,0,0,1,0,0,0,17.16,17.159999
3,10.00,4,0.348554,0,0,0,0,0,1,0,0,0,28.69,29.611726
4,5.91,4,0.205995,0,0,0,0,0,1,0,0,0,28.69,28.690001


In [31]:
# rename Expenditure_per_wk to target
regressor_reference.rename({
    'Expenditure_per_wk':'target'
},
    axis =1,
    inplace=True
)

regressor_reference.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,target,prediction
0,3.29,3,0.191725,0,0,0,0,0,1,0,0,0,17.16,18.844135
1,10.89,3,0.634615,0,0,0,0,0,1,0,0,0,17.16,21.880973
2,2.98,3,0.173660,0,0,0,0,0,1,0,0,0,17.16,17.159999
3,10.00,4,0.348554,0,0,0,0,0,1,0,0,0,28.69,29.611726
4,5.91,4,0.205995,0,0,0,0,0,1,0,0,0,28.69,28.690001


In [32]:
# rename Expenditure_per_wk to target
regressor_curr_dt_no_drift.rename({
    'Expenditure_per_wk':'target'
},
    axis =1,
    inplace=True
)

regressor_curr_dt_no_drift.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,target,prediction
0,10.00,10,0.144009,1,6,0,0,0,0,0,1,0,69.44,69.381661
1,8.27,3,0.226948,3,9,0,0,0,0,1,0,0,36.44,36.440000
2,1.60,1,1.000000,0,0,0,0,0,0,0,0,0,1.60,1.599999
3,6.84,11,0.106992,7,6,1,0,0,0,0,3,0,63.93,63.855861
4,4.93,9,0.084752,0,1,0,1,0,0,0,1,0,58.17,58.170015


In [33]:

regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=regressor_reference.sort_index(), current_data=regressor_curr_dt_no_drift.sort_index())
regression_performance_report.save_html("../11. Reports/evidently_metric_presets/model_performance/regressor_curr_dt_no_drift_model_performance_report29082023.html")


In [34]:
# rename Expenditure_per_wk to target
regressor_drift.rename({
    'Expenditure_per_wk':'target'
},
    axis =1,
    inplace=True
)

regressor_drift.head()

,Total_Price,Nbr_trips_per_wk,Total_Exp_wk_perc,Raw_meats_wk,Vegetables_wk,Cooking_base_wk,Breakfast_wk,Transport_wk,Electronics_wk,Education_wk,Cosmetics_and_selfcare_wk,Clothes_and_shoes_wk,target,prediction
0,29.671353,27.339236,0.182824,21.605707,22.686535,14.965869,11.628342,11.374895,8.010144,17.332147,10.617962,3.965802,81.338213,103.036164
1,10.772184,16.613967,0.282498,5.329787,13.760860,6.712280,11.391198,5.400155,12.412875,11.662304,13.654401,15.852365,47.051797,133.756120
2,2.789794,8.747711,1.065763,14.982056,10.591974,4.179518,2.854285,13.613707,8.852314,9.874487,5.014989,14.859072,23.460449,113.995778
3,19.297759,29.517343,0.193383,20.215022,7.217432,5.672266,4.964222,9.193924,18.293691,6.078218,8.594319,14.789202,79.541401,118.684781
4,15.021405,30.802891,0.093229,5.633105,10.219041,6.554663,17.109600,6.630360,5.371829,8.588563,13.757232,21.658638,82.716934,119.798779


In [35]:

regression_performance_report = Report(metrics=[
    RegressionPreset(),
])

regression_performance_report.run(reference_data=regressor_reference.sort_index(), current_data=regressor_drift.sort_index())
regression_performance_report.save_html("../11. Reports/evidently_metric_presets/model_performance/regressor_curr_dt_with_drift_model_performance_report29082023.html")
